<a href="https://colab.research.google.com/github/mcdaqc/gpt2-offline-android-app/blob/master/FIne_Tuning_a_GPT2_de_KerasNLP_para_Generaci%C3%B3n_de_Texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generacion de texto con GPT2 de KerasNLP

**Author:** David Alonso Quispe Castillo<br>
**Date created:** 14/07/2023<br>
**Last modified:** 14/07/2023<br>
**Objetivo:** Usar el modelo GPT-2 de KerasNLP y los generadores de texto (samplers) para la generación de texto.

En este proyecto, basado en el trabajo previo de [Chen Qian](https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/generative/ipynb/gpt2_text_generation_with_kerasnlp.ipynb#scrollTo=76gl9HSIF-uB), utilizaremos [KerasNLP](https://keras.io/keras_nlp/) para cargar un modelo de lenguaje grande  (LLM) pre-entrenado - el modelo [GPT-2 ](https://openai.com/research/better-language-models), desarrollado por OpenAI. Aprovecharemos las capacidades de generación de texto de este modelo para crear texto basado en una entrada proporcionada por el usuario. Además, se demostrará cómo GPT-2 puede adaptarse rápidamente a otros idiomas, como el español.

##  Antes de comenzar

Colab ofrece diferentes tipos de entornos de ejecución. Asegúrate de ir a **Entorno de ejecución -> Cambiar tipo de entorno de ejecución** y seleccionar el entorno de ejecución con aceleración de hardware GPU (que debería tener >12 GB de RAM del host y ~15 GB de RAM de la GPU) ya que ajustaremos finamente el modelo GPT-2. Ejecutar este tutorial en un entorno de ejecución de CPU tomaría horas.

## Instalamos KerasNLP e Importamos Dependencias

In [ ]:
!pip install -q keras-nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 576.5/576.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.0/728.0 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 96.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 47.1 MB/s eta 0:00:00


In [ ]:
import keras_nlp
import tensorflow as tf
from tensorflow import keras
import time

Using TensorFlow backend


## Introducción a los Modelos de Lenguaje Generativos de Gran Escala (LLMs)

Los Modelos de Lenguaje Grande (LLMs, por sus siglas en inglés) son un tipo de modelos de aprendizaje automático que se entrenan en un gran corpus de datos de texto para generar resultados en varias tareas de Procesamiento del Lenguaje Natural (NLP, por sus siglas en inglés), como generación de texto, respuesta a preguntas o traducción automática.

Los LLMs generativos se basan típicamente en redes neuronales de aprendizaje profundo, como la arquitectura [Transformer](https://arxiv.org/abs/1706.03762) inventada por investigadores de Google en 2017, y se entrenan con grandes cantidades de datos de texto, a menudo involucrando miles de millones de palabras. Estos modelos, como [LaMDA](https://blog.google/technology/ai/lamda/) y [PaLM](https://ai.googleblog.com/2022/04/pathways-language-model-palm-scaling-to.html) de Google, se entrenan con conjuntos de datos extensos provenientes de diversas fuentes de datos, lo que les permite generar resultados para múltiples tareas. El núcleo de los LLMs generativos consiste en predecir la siguiente palabra en una oración, a menudo conocido como **Preentrenamiento LM Causal**. De esta manera, los LLMs pueden generar texto coherente basado en las indicaciones del usuario. Para obtener una discusión más pedagógica sobre los modelos de lenguaje, puedes consultar la [clase de Stanford CS324 LLM](https://stanford-cs324.github.io/winter2022/lectures/introduction/).

## Introducción a KerasNLP

La construcción y entrenamiento de Modelos de Lenguaje Grande (LLMs) desde cero son complejos y costosos. Afortunadamente, existen LLMs pre-entrenados disponibles para su uso inmediato. [KerasNLP](https://keras.io/keras_nlp/) proporciona una amplia variedad de puntos de control pre-entrenados que te permiten experimentar con modelos de última generación sin necesidad de entrenarlos desde cero.

KerasNLP es una biblioteca de procesamiento del lenguaje natural que brinda soporte a los usuarios a lo largo de todo su ciclo de desarrollo. KerasNLP ofrece tanto modelos pre-entrenados como componentes modulares, lo que permite a los desarrolladores reutilizar fácilmente los modelos pre-entrenados o construir sus propios LLM.

En pocas palabras, para los LLMs generativos, KerasNLP ofrece:

- Modelos pre-entrenados con el método `generate()`, por ejemplo en, `keras_nlp.models.GPT2CausalLM` y `keras_nlp.models.OPTCausalLM`.
- Clase de muestreo (Sampler) que implementa algoritmos de generación como Top-K, Beam y búsqueda contrastiva. Estos samplers se pueden utilizar para generar texto con modelos personalizados.



## Cargamos un modelo pre-entrenado de GPT-2 y generamos texto

KerasNLP proporciona varios modelos pre-entrenados, como [Google
Bert](https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html)
y [GPT-2](https://openai.com/research/better-language-models). Puedes ver
la lista de modelos disponibles en el [repositorio de KerasNLP](https://github.com/keras-team/keras-nlp/tree/master/keras_nlp/models).

Es muy fácil cargar el modelo GPT-2, como puedes ver a continuación:

In [ ]:
# Para acelerar el entrenamiento y la generación, utilizamos un preprocesador
# de longitud 128 en lugar de la longitud completa de 1024.

preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en", preprocessor=preprocessor
)

Una vez que el modelo está cargado, puedes usarlo para generar texto de inmediato. Ejecuta las celdas a continuación para probarlo. Es tan simple como llamar a la función generate() una sola vez.

In [ ]:
start = time.time()

output = gpt2_lm.generate("My trip to Yosemite was", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
My trip to Yosemite was a success. It's been my passion since I was a kid and I love Yosemite.

The first day was great because I was able to see a few things in Yosemite.

I got to take a few photos of the beautiful landscape. The first day is great because I was able to take a few photos of the beautiful landscape.

I had to go to a different place to take a few pictures.

I had to go to a different place to take a few pictures.

I had to go to a different place to take a few pictures.

I didn't get to spend time at all in Yosemite.

It was so cold out and the weather wasn't that great.

I had to go to another place to get a little bit of sun.

There was a lot of rain in the area, but I was so excited about it.

I got to see a few of the
TOTAL TIME ELAPSED: 27.92s


Intenta con otro ejemplo:

In [ ]:
start = time.time()

output = gpt2_lm.generate("That Italian restaurant is", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
That Italian restaurant is a classic. It has the most beautiful and most delicious food in the world. It is the perfect place to have your family.

If you like to eat here and have a good time, then this is the place to go!

The place is so small that it is impossible to tell where it is. It looks like it is a restaurant but there are a lot of other things in the restaurant.

The menu looks pretty good and the food is really good. I have had a couple of different types of food here, so I would recommend it.

The food is very good and I love that it is made with real Italian ingredients. The food is so good I have to go back to it.

I love this restaurant.

This is the best place in the world. The food is very good. I have never been to this place. It is a great place to get a good meal.

I love
TOTAL TIME ELAPSED: 2.05s


Observa cuánto más rápido es el segundo llamado. Esto se debe a que el grafo computacional se compila con [XLA](https://www.tensorflow.org/xla) en la primera ejecución y se reutiliza en la segunda, en segundo plano.

La calidad del texto generado parece estar bien, pero podemos mejorarlo mediante el ajuste fino / afinación (fine-tuning.).

## Más sobre el modelo GPT-2 de KerasNLP

A continuación, vamos a ajustar finamente el modelo para actualizar sus parámetros, pero antes de hacerlo, echemos un vistazo al conjunto completo de herramientas que tenemos para trabajar con GPT2.

El código de GPT2 se puede encontrar [aquí](https://github.com/keras-team/keras-nlp/blob/master/keras_nlp/models/gpt2/). Conceptualmente, `GPT2CausalLM` se puede descomponer jerárquicamente en varios módulos en KerasNLP, todos los cuales tienen una función *from_preset()* que carga un modelo pre-entrenado:

- `keras_nlp.models.GPT2Tokenizer`: El tokenizador utilizado por el modelo GPT2, que es un [codificador de pares de bytes](https://huggingface.co/course/chapter6/5?fw=pt).
- `keras_nlp.models.GPT2CausalLMPreprocessor`: el preprocesador utilizado en el entrenamiento causal de GPT2. Realiza la tokenización junto con otros trabajos de preprocesamiento, como la creación de la etiqueta y la adición del token de finalización.
- `keras_nlp.models.GPT2Backbone`: el modelo GPT2, que es una pila de `keras_nlp.layers.TransformerDecoder`. A esto generalmente se le denomina simplemente `GPT2`.
- `keras_nlp.models.GPT2CausalLM`: envuelve a `GPT2Backbone`, multiplica la salida de `GPT2Backbone` por la matriz de embedding para generar logits sobre los tokens del vocabulario.

## Ajuste fino (Fine-tuning) en el conjunto de datos WikiHow

Ahora que tienes conocimientos sobre el modelo GPT-2 de KerasNLP, puedes dar un paso más para ajustar finamente el modelo y lograr que genere texto en un estilo específico, ya sea corto o largo, formal o casual. En este tutorial, utilizaremos el conjunto de datos de Wikihow como ejemplo.



## Finetune en Dataset Español

También podemos ajustar finamente GPT2 en conjuntos de datos que no sean su idioma principal de entrenamiento. Esta parte muestra cómo ajustar finamente GPT2 en un conjunto de datos de wikihow en español para enseñar a nuestro modelo a convertirse en un sabelotodo.

Debido a que GPT2 utiliza un codificador de pares de bytes (byte-pair encoder) y el conjunto de datos de pre-entrenamiento original contiene algunos caracteres en español, podemos utilizar el vocabulario original para ajustar finamente en un conjunto de datos en español.

In [ ]:
!# Descargamos el dataset desde HuggingFace
git clone https://huggingface.co/daqc/wikihow-spanish

Echemos un vistazo dentro de los datos de muestra del conjunto de datos:


In [ ]:
import json

# Leer el archivo JSON
with open('dataset/spanish.json', 'r') as file:
    data = json.load(file)

# Obtener la primera clave y el primer elemento del diccionario
primer_clave = next(iter(data))
primer_elemento = data[primer_clave]

# Imprimir la clave y la estructura del primer elemento
print("Clave:", primer_clave)
print("Estructura:")
print(json.dumps(primer_elemento, indent=4))

Clave: https://es.wikihow.com/calcular-el-rendimiento-anualizado-de-una-cartera-de-inversiones
Estructura:
{
    "Calcular tu rendimiento anualizado": {
        "summary": "Calcula tu rendimiento anualizado. Calcula el rendimiento semestral. Calcula un equivalente anualizado.",
        "document": "Una vez que hayas calculado el rendimiento total (como se muestra arriba), ingresa el resultado en esta ecuaci\u00f3n: rendimiento anualizado = (1+ rendimiento)1/N-1 El producto de esta ecuaci\u00f3n ser\u00e1 el n\u00famero correspondiente al rendimiento de cada a\u00f1o durante todo el periodo de tiempo.  En el exponente (el n\u00famero peque\u00f1o que est\u00e1 afuera del par\u00e9ntesis), el \u201c1\u201d representa la unidad que estamos midiendo, que es un a\u00f1o. Si deseas ser m\u00e1s espec\u00edfico, podr\u00edas usar \u201c365\u201d para obtener el rendimiento diario. La \u201cN\u201d representa el n\u00famero de periodos que medir\u00e1s. Entonces, si mides tu rendimiento en 7 a

En nuestro caso, al realizar la predicción de la siguiente palabra en un modelo de lenguaje, necesitamos obtener solamente la informacion de relevancia del dataset mediante el siguiente codigo:

In [ ]:
import os
import json
from urllib.parse import urlsplit, unquote

wikihow_data = []
for file in os.listdir("dataset/"):
    if ".json" not in file:
        continue
    full_filename = os.path.join("dataset/", file)
    with open(full_filename, "r") as f:
        content = json.load(f)
        wikihow_data.append(content)

paragraphs = []
for parent_url, sections in data.items():
    decoded_parent_url = "¿Cómo " + unquote(urlsplit(parent_url).path.split("/")[-1]).replace('-', ' ') + "?"
    paragraph = f"{decoded_parent_url}\n"
    for section in sections:
        section_data = sections[section]
        title = section + ":"
        summary = section_data.get("summary", "")
        document = section_data.get("document", "")
        paragraph += f"{title}\n{summary}\n{document}\n"
    paragraphs.append(paragraph)

print("Parrafo 1:")
print(paragraphs[0])
print()
#print("Parrafo 2:")
#print(paragraphs[1])


Parrafo 1:
¿Cómo calcular el rendimiento anualizado de una cartera de inversiones?
Calcular tu rendimiento anualizado:
Calcula tu rendimiento anualizado. Calcula el rendimiento semestral. Calcula un equivalente anualizado.
Una vez que hayas calculado el rendimiento total (como se muestra arriba), ingresa el resultado en esta ecuación: rendimiento anualizado = (1+ rendimiento)1/N-1 El producto de esta ecuación será el número correspondiente al rendimiento de cada año durante todo el periodo de tiempo.  En el exponente (el número pequeño que está afuera del paréntesis), el “1” representa la unidad que estamos midiendo, que es un año. Si deseas ser más específico, podrías usar “365” para obtener el rendimiento diario. La “N” representa el número de periodos que medirás. Entonces, si mides tu rendimiento en 7 años, tendrás que utilizar este número en lugar de "N". Por ejemplo, supongamos que en un periodo de siete años, el valor de tu cartera creció de $1,000 a $2,500. Primero calcula tu r

Damos un vistazo a los datos de ejemplo.

In [ ]:
print(paragraphs[1])

¿Cómo envolver una cinta alrededor de una caja?
Hacer un moño en diagonal:
Pasa la cinta por la esquina superior izquierda de la caja. Pasa la cinta detrás de la esquina superior derecha. Pasa la cinta por la esquina inferior derecha y debajo de la esquina inferior izquierda. Lleva la cinta hacia la esquina superior izquierda. Corta el exceso de cinta. Cruza y amarra las cintas. Cortar el exceso de cinta.
Deja alrededor de 10 a 20 centímetros (4 a 8 pulgadas) colgando de la esquina del lado izquierdo. Deja el resto de la cinta en el carrete en el borde superior. Toma el lado del carrete de la cinta y pásala por detrás de la esquina superior derecha, hacia la esquina inferior derecha. Mantén el pulgar sobre la cinta en la esquina superior izquierda para que no se caiga. Mantén las cintas bien colocadas y ceñidas para que no se salgan de las esquinas. En este punto, sería una buena idea que ajustes las posiciones de las cintas envueltas en cada una de las esquinas. Si parece que se están

Now you can finetune the model using the familiar *fit()* function. Note that
`preprocessor` will be automatically called inside `fit` method since
`GPT2CausalLM` is a `keras_nlp.models.Task` instance.

This step takes quite a bit of GPU memory and a long time if we were to train
it all the way to a fully trained state. Here we just use part of the dataset
for demo purposes.

In [ ]:
import tensorflow_datasets as tfds
import tensorflow.keras as keras
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Preparar el conjunto de datos
train_ds = (
    tf.data.Dataset.from_tensor_slices(paragraphs)
    .batch(16)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)


# Definir el número de épocas a 1 para demostración
num_epochs = 3

# Directorio para guardar los checkpoints
checkpoint_dir = "checkpoints/"

# Callback para guardar checkpoints por epoch
checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, "model_{epoch:02d}.h5"),
    save_freq="epoch",
    save_weights_only=True,
    save_best_only=False,
    verbose=1
)

# Verificar si hay checkpoints existentes para continuar desde el último
checkpoint_files = [file for file in os.listdir(checkpoint_dir) if file.endswith('.h5')]
if checkpoint_files:
    latest_checkpoint = max(checkpoint_files)
    gpt2_lm.load_weights(os.path.join(checkpoint_dir, latest_checkpoint))

    # Obtener el número de epoch desde el nombre del archivo de checkpoint
    num_epoch_resume = int(latest_checkpoint.split("_")[1].split(".")[0])
    #num_epochs -= num_epoch_resume  # Restar las épocas ya entrenadas

    # Entrenar el modelo
    history = gpt2_lm.fit(train_ds, initial_epoch=num_epoch_resume, epochs=num_epochs, callbacks=[checkpoint_callback])
    print("hola")
else:
    # No hay checkpoints existentes, entrenar el modelo desde cero
    # Entrenar el modelo
    history = gpt2_lm.fit(train_ds, epochs=num_epochs, callbacks=[checkpoint_callback])


2
1
Epoch 3/3
1/1 [==============================] - ETA: 0s - loss: 4.7384 - accuracy: 0.1947
Epoch 3: saving model to checkpoints/model_03.h5
1/1 [==============================] - 26s 26s/step - loss: 4.7384 - accuracy: 0.1947
hola



##  Métricas y parámetros para evaluar el rendimiento y la eficiencia durante el entrenamiento.

Una vez terminado el entrenamiento podemos visualizar y monitorear el rendimiento del modelo durante el entrenamiento, proporcionando métricas como pérdida, precisión, tasa de aprendizaje y eficiencia computacional.

In [ ]:

# Obtener las métricas de pérdida, precisión y tasa de aprendizaje durante el entrenamiento
loss = history.history['loss']
accuracy = history.history['accuracy'] # Reemplaza 'accuracy' con 'weighted_accuracy' si estás utilizando esta métrica
learning_rate = np.array([learning_rate(step) for step in range(len(history.epoch))])

# Obtener la eficiencia computacional
time_per_epoch = history.epoch[-1] / len(history.epoch)  # Tiempo por época
efficiency = [time_per_epoch * (i + 1) for i in range(num_epochs)]

# Graficar la pérdida, precisión, tasa de aprendizaje y eficiencia
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(12, 4))

plt.subplot(1, 4, 1)
plt.plot(epochs, loss, 'b', label='Pérdida')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Valor')
plt.legend()

plt.subplot(1, 4, 2)
plt.plot(epochs, accuracy, 'r', label='Precisión')
plt.title('Precisión durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Valor')
plt.legend()

plt.subplot(1, 4, 3)
plt.plot(epochs, learning_rate, 'g', label='Tasa de Aprendizaje')
plt.title('Tasa de Aprendizaje durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Valor')
plt.legend()

plt.subplot(1, 4, 4)
plt.plot(epochs, efficiency, 'm', label='Eficiencia')
plt.title('Eficiencia Computacional durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Tiempo (segundos)')
plt.legend()

plt.tight_layout()
plt.show()


TypeError: ignored

Una vez finalizado el ajuste fino, puedes generar texto nuevamente utilizando la misma función generate(). Esta vez, el texto estará más cerca del estilo de wikihow y la longitud generada será similar a la longitud preestablecida en el conjunto de entrenamiento.

In [ ]:
start = time.time()


output = gpt2_lm.generate("Cepilla a tu burro regularmente", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TIEMPO TOTAL TRANSCURRIDO: {end - start:.2f}s")

## Acerca del Método de Muestreo

En KerasNLP, ofrecemos varios métodos de muestreo, como la búsqueda contrastiva, el muestreo Top-K y el muestreo de haz (beam sampling). Por defecto, nuestro `GPT2CausalLM` utiliza el muestreo Top-K, pero puedes elegir tu propio método de muestreo.

Al igual que con el optimizador y las funciones de activación, hay dos formas de especificar tu propio muestreador personalizado:

- Utilizar un identificador de cadena, como "greedy", si deseas utilizar la configuración predeterminada de esta forma.
- Pasar una instancia de `keras_nlp.samplers.Sampler`, si deseas utilizar una configuración personalizada de esta forma.

In [ ]:
# Use a string identifier.
gpt2_lm.compile(sampler="top_k")
output = gpt2_lm.generate("Cepilla a tu burro regularmente", max_length=200)
print("\nGPT-2 output:")
print(output)

# Use a `Sampler` instance. `GreedySampler` tends to repeat itself,
greedy_sampler = keras_nlp.samplers.GreedySampler()
gpt2_lm.compile(sampler=greedy_sampler)

output = gpt2_lm.generate("Cepilla a tu burro regularmente", max_length=200)
print("\nGPT-2 output:")
print(output)


GPT-2 output:
I like basketball, so it's not that I can't play. But I can't just do it. I can't do it in the same way as anybody else.
"But it's not a bad thing. You have to do it in a way that you can't do it without being a bad person. You have to be a bad person, but you have to be a good person, too."
I don't want to be bad. But I don't want to become a bad person. I don't want to be a person who doesn't play. I don't want to be a bad person.
"I don't want to be bad."
I don't want to be a bad person.
I don't want to be a bad person.
And I won't let that happen.



GPT-2 output:
I like basketball. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to play it. I like to


Para obtener más detalles sobre la clase `Sampler` de KerasNLP, puedes revisar el código [aquí](https://github.com/keras-team/keras-nlp/tree/master/keras_nlp/samplers).

## Guarda tu Modelo


*** Guarda el modelo: Utiliza el método model.save() de Keras para guardar todo el modelo, incluyendo la configuración y los pesos, en un archivo. Especifica una ruta y un nombre de archivo para guardar el modelo

In [ ]:
model.save("saved_model_h5/model.h5")

 Si estás trabajando principalmente con TensorFlow y planeas utilizar el modelo en otros proyectos o plataformas compatibles con SavedModel, entonces tf.saved_model.save() puede ser más conveniente.

In [ ]:
saved_model_dir = "saved_model/"
tf.saved_model.save(gpt2_lm, saved_model_dir)